In [1]:
import os
import sys
from collections import OrderedDict

In [2]:
sys.path.append("../..")

In [3]:
import spatialpy
spatialpy.__file__

'../../spatialpy/__init__.py'

In [4]:
import matplotlib.pyplot as plt
import numpy

# Global Constants
MAX_X_DIM = 5.0
MIN_X_DIM = -5.0
TOL = 1e-9

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
class Edge1(spatialpy.Geometry):
    def inside(self, x, on_boundary):
        return abs(x[0] - MAX_X_DIM) < 0.05
class Edge2(spatialpy.Geometry):
    def inside(self, x, on_boundary):
        return abs(x[0] - MIN_X_DIM) < 0.05
class Middle(spatialpy.Geometry):
    def inside(self, x, on_boundary):
        return abs(x[0] - MIN_X_DIM) >= 0.05

In [7]:
class cylinderDemo3D(spatialpy.Model):
    def __init__(self, model_name="cylinder_demo3d"):
        spatialpy.Model.__init__(self, model_name)

        self.timestep_size = 1e-3
        
        # System constants
        D_const = 0.1

        # Define Species
        A = spatialpy.Species(name="A", diffusion_constant=D_const)
        B = spatialpy.Species(name="B", diffusion_constant=D_const)
        self.add_species([A, B])

        # Define Geometry
        self.domain = spatialpy.Domain.read_xml_mesh('cylinder.xml')

        # Define Subdomains
        self.set_type(Middle(), 1)
        self.set_type(Edge1(),  2)
        self.set_type(Edge2(),  3)
        
        # Restrict the movement of Chemical Species
        self.restrict(A,[1,2])
        self.restrict(B,[1,3])

        vol = self.domain.get_vol()
        print("vol",vol)
        type = self.domain.type
        left = numpy.sum(vol[type == 2])
        right = numpy.sum(vol[type == 3])
        print("left "+str(left)+" right "+str(right))
        
        k_react = spatialpy.Parameter(name="k_react", expression=1.0)
        k_creat1 = spatialpy.Parameter(name="k_creat1", 
                                     expression=100/left)
        k_creat2 = spatialpy.Parameter(name="k_creat2", 
                                     expression=100/right)
        self.add_parameter([k_react, k_creat1,k_creat2])


        # Define Reactions
        R1 = spatialpy.Reaction(reactants=None, products={A:1}, 
                                rate=k_creat1, restrict_to=2)
        R2 = spatialpy.Reaction(reactants=None, products={B:1}, 
                              rate=k_creat2, restrict_to=3)
        R3 = spatialpy.Reaction(reactants={A:1, B:1}, products=None, 
                              rate=k_react)
        self.add_reaction([R1, R2, R3])
        
        self.timespan(range(500))

In [8]:
model = cylinderDemo3D()

vol [0.01513526 0.07034112 0.02382667 ... 0.01674217 0.02120607 0.01969156]
left 0.5092013833059308 right 0.505804729089437


In [9]:
print(f"Number of Timesteps: {model.num_timesteps}")
print(f"Output Frequency: {model.output_freq}")
print(f"Timestep Size: {model.timestep_size}")
%time results = model.run()

Number of Timesteps: 499000
Output Frequency: 1000
Timestep Size: 0.001
CPU times: user 51.6 ms, sys: 28 ms, total: 79.6 ms
Wall time: 39min 59s


In [15]:
model.timestep_size = 1
model.timespan(range(500))
print(f"Number of Timesteps: {model.num_timesteps}")
print(f"Output Frequency: {model.output_freq}")
print(f"Timestep Size: {model.timestep_size}")
%time results2 = model.run()

Number of Timesteps: 499
Output Frequency: 1
Timestep Size: 1
CPU times: user 19 ms, sys: 12.2 ms, total: 31.2 ms
Wall time: 34.6 s


# Visualization

In [16]:
def analyze_data(results, t_ndx=0):
    _, data = results.read_step(step_num=t_ndx)
    
    import plotly.graph_objects as go

    concentration_data = data["D[A]"] / (data['mass'] / data['rho'])

    fig = go.Figure()
    fig.add_trace(go.Histogram(x=data["C[A]"]))
    fig.add_trace(go.Histogram(x=data["D[A]"]))
    fig.add_trace(go.Histogram(x=concentration_data))

    fig.update_layout(
        xaxis_title_text='Value', # xaxis label
        yaxis_title_text='Count', # yaxis label
        bargap=0.2, # gap between bars of adjacent location coordinates
        bargroupgap=0.1 # gap between bars of the same location coordinates
    )
    fig.show()

### Time Step Size = 1e-3

In [23]:
analyze_data(results=results, t_ndx=200000)

In [24]:
results.plot_species('A', t_ndx=200000, width=None, height=None, title="Raw Stochastic Results")
results.plot_species('A', t_ndx=200000, concentration=True, width=None, height=None, title="Stochastic Results")
results.plot_species('A', t_ndx=200000, deterministic=True, width=None, height=None, title="Deterministic Results")

### Time Step Size = 1

In [25]:
analyze_data(results=results2, t_ndx=200)

In [26]:
results2.plot_species('A', t_ndx=200, width=None, height=None, title="Raw Stochastic Results")
results2.plot_species('A', t_ndx=200, concentration=True, width=None, height=None, title="Stochastic Results")
results2.plot_species('A', t_ndx=200, deterministic=True, width=None, height=None, title="Deterministic Results")